In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#use the citipy module to determine city based on latitude and longitude
from citipy import citipy
#import the datetime module from the datetime library
from datetime import datetime
#for API requests
import requests
#import the API key
from config import weather_api_key
# Import the time module.
import time
# Import linear regression from the SciPy stats module
from scipy.stats import linregress
import csv
import os
import gmaps


In [2]:
#file to load
weather_data_to_load ="weather_data/WeatherPy_challenge.csv"


In [3]:
#read csv files into a df
weather_data_df=pd.read_csv(weather_data_to_load)
weather_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description,Rain (last 3 hrs),Snow (last 3 hrs)
0,0,Ushuaia,AR,2020-04-19 01:11:30,-54.80,-68.30,48.20,57,40,10.29,scattered clouds,0,0
1,1,Punta Arenas,CL,2020-04-19 01:08:30,-53.15,-70.92,46.40,65,40,19.46,scattered clouds,0,0
2,2,Bredasdorp,ZA,2020-04-19 01:09:26,-34.53,20.04,46.40,93,68,3.36,broken clouds,0,0
3,3,Mataura,NZ,2020-04-19 01:11:32,-46.19,168.86,55.99,65,67,8.99,broken clouds,0,0
4,4,Vardo,NO,2020-04-19 01:09:20,70.37,31.11,36.88,81,98,18.16,overcast clouds,0,0
5,5,Port Alfred,ZA,2020-04-19 01:09:05,-33.59,26.89,62.01,86,0,5.01,clear sky,0,0
6,6,Kodiak,US,2020-04-19 01:11:33,57.79,-152.41,46.40,75,40,9.17,scattered clouds,0,0
7,7,Hobart,AU,2020-04-19 01:07:55,-42.88,147.33,55.99,71,75,5.82,broken clouds,0,0
8,8,Touros,BR,2020-04-19 01:11:34,-5.20,-35.46,78.80,94,75,2.24,broken clouds,0,0
9,9,Markova,UA,2020-04-19 01:11:34,48.68,24.41,37.56,62,49,0.87,scattered clouds,0,0


In [4]:
weather_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current description     object
Rain (last 3 hrs)        int64
Snow (last 3 hrs)        int64
dtype: object

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_req = str(input("Do you want it to be raining? (yes/no) "))
snow_req = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
#filter if rain_req  
if rain_req == "yes":
    preferred_weather_df1 = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & (weather_data_df["Max Temp"] >= min_temp) & (weather_data_df["Rain (last 3 hrs)"]>0)  ]
else:
    preferred_weather_df1 = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & (weather_data_df["Max Temp"] >= min_temp)   ]
#filter if snow_req
if snow_req == "yes":
    preferred_weather_df2 = preferred_weather_df1.loc[(preferred_weather_df1["Max Temp"] <= max_temp) & (preferred_weather_df1["Max Temp"] >= min_temp) & (preferred_weather_df1["Snow (last 3 hrs)"]>0)  ]
else:
    preferred_weather_df2 = preferred_weather_df1.loc[(preferred_weather_df1["Max Temp"] <= max_temp) & (preferred_weather_df1["Max Temp"] >= min_temp)   ]

preferred_weather_df2

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description,Rain (last 3 hrs),Snow (last 3 hrs)
8,8,Touros,BR,2020-04-19 01:11:34,-5.20,-35.46,78.80,94,75,2.24,broken clouds,0,0
13,13,Affery,CI,2020-04-19 01:11:37,6.32,-3.96,75.83,94,94,2.86,overcast clouds,0,0
14,14,Bilma,NE,2020-04-19 01:11:38,18.69,12.92,82.87,13,6,6.38,clear sky,0,0
15,15,Kahului,US,2020-04-19 01:08:43,20.89,-156.47,84.20,51,20,16.11,few clouds,0,0
16,16,Buala,SB,2020-04-19 01:11:39,-8.14,159.59,83.80,71,51,2.24,broken clouds,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,549,La Primavera,CO,2020-04-19 01:17:00,5.49,-70.41,80.67,73,73,4.45,moderate rain,0,0
552,552,Beira,MZ,2020-04-19 01:17:05,-19.84,34.84,75.13,84,12,12.15,light rain,0,0
559,559,Labelle,US,2020-04-19 01:16:48,26.76,-81.44,80.60,69,1,4.70,clear sky,0,0
563,563,Vila Velha,BR,2020-04-19 01:17:11,-20.33,-40.29,75.20,78,75,11.41,broken clouds,0,0


In [7]:
#create the output file (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"
#export the challenge weather data into a CSV
preferred_weather_df2.to_csv(output_data_file, index_label="City_ID")

In [15]:
import gmaps
# Import the API key.
from config import g_key
gmaps.configure(api_key=g_key)

#add heatmap hotel, city, country, desc
locations = preferred_weather_df2[["Lat", "Lng"]]
#max_temp = preferred_weather_df2["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#call the figure to plot data
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
#create dataframe hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_weather_df2[["City", "Current description","Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Current description,Country,Max Temp,Lat,Lng,Hotel Name
8,Touros,broken clouds,BR,78.80,-5.20,-35.46,
13,Affery,overcast clouds,CI,75.83,6.32,-3.96,
14,Bilma,clear sky,NE,82.87,18.69,12.92,
15,Kahului,few clouds,US,84.20,20.89,-156.47,
16,Buala,broken clouds,SB,83.80,-8.14,159.59,
...,...,...,...,...,...,...,...
549,La Primavera,moderate rain,CO,80.67,5.49,-70.41,
552,Beira,light rain,MZ,75.13,-19.84,34.84,
559,Labelle,clear sky,US,80.60,26.76,-81.44,
563,Vila Velha,broken clouds,BR,75.20,-20.33,-40.29,


In [10]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

#iterate through dataframe
for index, row in hotel_df.iterrows():
    #get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    #add the latitude and longitude to location key for the params dictionary in gmaps
    params["location"] = f"{lat},{lng}"

    
     # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
    
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df

,City,Current description,Country,Max Temp,Lat,Lng,Hotel Name
8,Touros,broken clouds,BR,78.80,-5.20,-35.46,INN NEW HORIZON
13,Affery,overcast clouds,CI,75.83,6.32,-3.96,
14,Bilma,clear sky,NE,82.87,18.69,12.92,Expeditions Ténére voyages
15,Kahului,few clouds,US,84.20,20.89,-156.47,Maui Seaside Hotel
16,Buala,broken clouds,SB,83.80,-8.14,159.59,Maringe Lagoon Lodge
...,...,...,...,...,...,...,...
549,La Primavera,moderate rain,CO,80.67,5.49,-70.41,La Riviera Hotel Colonial
552,Beira,light rain,MZ,75.13,-19.84,34.84,Beira Terrace Hotel
559,Labelle,clear sky,US,80.60,26.76,-81.44,La Belle Motel
563,Vila Velha,broken clouds,BR,75.20,-20.33,-40.29,Hotel Vitória Palace


In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current description} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))